In [193]:
!pip install lazypredict

In [194]:
from lazypredict.Supervised import LazyClassifier,LazyRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import RidgeCV

In [195]:
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [196]:
file_match_1 = open('/content/match_1.json')
data = json.load(file_match_1)
df_match_1=pd.DataFrame(data)
df_match_1

,label,norm
0,walk,"[23.177037336396975, 23.36152528524938, 21.535..."
1,walk,"[20.998214383911275, 19.182798059840767, 18.27..."
2,walk,"[21.95259682019565, 20.62720484424047, 22.5554..."
3,walk,"[19.39209748358647, 19.30460665293087, 18.9787..."
4,rest,"[22.069263037713093, 19.672270483203395, 19.88..."
...,...,...
569,walk,"[21.245737438782683, 21.938836193196945, 21.24..."
570,walk,"[20.272347748199333, 21.820302584302983, 21.21..."
571,run,"[90.21306005395827, 72.12712844635696, 63.5643..."
572,tackle,"[44.79510251441832, 27.26406867633046, 88.6164..."


In [197]:
#file_match_2 = open('/content/match_2.json')
#data_2 = json.load(file_match_2)
#df_match_2=pd.DataFrame(data_2)
#df_match_2

In [198]:
#df_match=pd.concat([df_match_1, df_match_2],ignore_index=True)
#df_match=df_match.reset_index()
df_match=df_match_1

## Missig values

In [199]:
df_match.isnull().sum()

label    0
norm     0
dtype: int64

In [200]:
df_match = df_match.dropna(subset=['label'])

In [201]:
def has_missing_values(lst):
    return any(x is None or np.isnan(x) for x in lst)
missing_values = df_match['norm'].apply(has_missing_values)

In [202]:
missing_values.any()

False

In [203]:
def interpolate_time_series(lst):
    series = pd.Series(lst)
    interpolated_series = series.interpolate(method='linear')
    return interpolated_series.tolist()
df_match['norm'] = df_match['norm'].apply(interpolate_time_series)

In [204]:
df_match.isnull().sum()

label    0
norm     0
dtype: int64

In [205]:
df_match

,label,norm
0,walk,"[23.177037336396975, 23.36152528524938, 21.535..."
1,walk,"[20.998214383911275, 19.182798059840767, 18.27..."
2,walk,"[21.95259682019565, 20.62720484424047, 22.5554..."
3,walk,"[19.39209748358647, 19.30460665293087, 18.9787..."
4,rest,"[22.069263037713093, 19.672270483203395, 19.88..."
...,...,...
569,walk,"[21.245737438782683, 21.938836193196945, 21.24..."
570,walk,"[20.272347748199333, 21.820302584302983, 21.21..."
571,run,"[90.21306005395827, 72.12712844635696, 63.5643..."
572,tackle,"[44.79510251441832, 27.26406867633046, 88.6164..."


In [206]:
df_match['label'].value_counts()

walk       245
run        209
dribble     42
rest        25
pass        24
tackle      20
shot         6
cross        3
Name: label, dtype: int64

## drop noaction

In [207]:
 df_match =df_match.drop(df_match[df_match['label'] == 'no action'].index)



In [208]:
df_match['label'].value_counts()

walk       245
run        209
dribble     42
rest        25
pass        24
tackle      20
shot         6
cross        3
Name: label, dtype: int64

## Feature engineering for regressor model

In [209]:
#df_match['avg']=df_match['norm'].apply(np.mean)
#df_match['std']=df_match['norm'].apply(np.std)
#df_match['max']=df_match['norm'].apply(np.max)
#df_match['min']=df_match['norm'].apply(np.min)
#df_match

In [210]:
df_match['sequence_length'] = df_match['norm'].apply(len)

In [211]:
 df_match =df_match.drop(df_match[(df_match['sequence_length'] < 10) | (df_match['sequence_length'] >300) ].index)

In [212]:
df_match.drop(columns='norm',inplace=True)


In [213]:
df_match

,label,sequence_length
0,walk,72
1,walk,54
2,walk,68
3,walk,75
4,rest,181
...,...,...
569,walk,85
570,walk,48
571,run,32
572,tackle,64


## label encoding

In [214]:
label_encoder= LabelEncoder()
label_encoder.fit(df_match['label'])
df_match['label'] = label_encoder.transform(df_match['label'])



In [215]:
label_encoder.classes_

array(['cross', 'dribble', 'pass', 'rest', 'run', 'shot', 'tackle',
       'walk'], dtype=object)

In [216]:
df_match

,label,sequence_length
0,7,72
1,7,54
2,7,68
3,7,75
4,3,181
...,...,...
569,7,85
570,7,48
571,4,32
572,6,64


In [217]:
df_match['precedent_label'] = df_match['label'].shift()
#df_match['precedent_sequence_length'] = df_match['sequence_length'].shift()
df_match = df_match.iloc[1:]
df_match['precedent_label'] = df_match['precedent_label'].astype(int)
#df_match['precedent_sequence_length'] = df_match['precedent_sequence_length'].astype(int)

In [218]:
df_match

,label,sequence_length,precedent_label
1,7,54,7
2,7,68,7
3,7,75,7
4,3,181,7
5,7,56,3
...,...,...,...
569,7,85,7
570,7,48,7
571,4,32,7
572,6,64,4


In [219]:
df_match['sample_temporal_index'] = df_match.index-1
#df_match

In [220]:

#quarter_length = len(df_match['sample_temporal_index']) // 4
#df_match['sample_temporal_index'] = pd.cut(df_match['sample_temporal_index'], bins=[0, quarter_length, 2*quarter_length, 3*quarter_length, len(df_match['sample_temporal_index'])], labels=[1, 2, 3, 4])
#

In [221]:
df_match

,label,sequence_length,precedent_label,sample_temporal_index
1,7,54,7,0
2,7,68,7,1
3,7,75,7,2
4,3,181,7,3
5,7,56,3,4
...,...,...,...,...
569,7,85,7,568
570,7,48,7,569
571,4,32,7,570
572,6,64,4,571


In [222]:
y=df_match['sequence_length']
x=df_match.drop(columns='sequence_length')

In [223]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2,random_state=42)


In [224]:
y_test

547    30
532    42
237    30
533    46
496    50
       ..
89     62
531    44
364    66
34     34
336    40
Name: sequence_length, Length: 114, dtype: int64

In [225]:
X_test

,label,precedent_label,sample_temporal_index
547,1,1,546
532,4,4,531
237,7,3,236
533,4,4,532
496,1,4,495
...,...,...,...
89,7,7,88
531,4,4,530
364,4,4,363
34,4,4,33


In [ ]:
clf = LazyRegressor()
try:
    models, predictions = clf.fit(X_train, X_test, y_train, y_test)
except Exception as e:
    print(f"An error occurred: {e}")


In [227]:
print(models)

                               Adjusted R-Squared  R-Squared    RMSE  \
Model                                                                  
PoissonRegressor                             0.16       0.18   17.04   
SGDRegressor                                 0.16       0.18   17.08   
RidgeCV                                      0.16       0.18   17.09   
Ridge                                        0.15       0.18   17.09   
TransformedTargetRegressor                   0.15       0.18   17.09   
LinearRegression                             0.15       0.18   17.09   
Lars                                         0.15       0.18   17.09   
LarsCV                                       0.15       0.18   17.09   
LassoLarsIC                                  0.15       0.18   17.09   
LassoLarsCV                                  0.15       0.18   17.09   
LassoCV                                      0.15       0.18   17.09   
BayesianRidge                                0.15       0.18   1

## Best regressor model is  based on and


In [228]:
regressor = RidgeCV(alphas=[1e-5,1e-4,1e-3, 1e-2, 1e-1, 1])

In [229]:
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
accuracy = regressor.score(X_test, y_test)

In [230]:
accuracy

0.17731052905888123

# if we do not drop samples with len<10 and len >300  we got 0.38 as accuracy wih model DecisionTreeRegressor

In [231]:
int(regressor.predict([[3,7,50]])[0])

46